In [ ]:
Install all the required libraries before you get started
# Install required Azure SDKs
# !pip install azure-iot-device 
#!pip azure-storage-blob 
#pip azure-mgmt-resource 
#pip azure-ai-machinelearning 
#pip azure-eventhub 
#pip  azure-keyvault azure-ai-openai


Import all required libraries before you get started
import os
from azure.iot.device import IoTHubDeviceClient, Message
from azure.storage.blob import BlobServiceClient
from azure.eventhub import EventHubProducerClient, EventData
from azure.ai.openai import OpenAIClient
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Job, AmlCompute, Input

print("######################")
print("Add all required environment variables")
print("######################")

AZURE_IOT_HUB_CONNECTION_STRING = os.getenv("AZURE_IOT_HUB_CONNECTION_STRING")
AZURE_BLOB_CONNECTION_STRING = os.getenv("AZURE_BLOB_CONNECTION_STRING")
AZURE_EVENT_HUB_CONNECTION_STRING = os.getenv("AZURE_EVENT_HUB_CONNECTION_STRING")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_STORAGE_CONTAINER = "sensor-data"

print("####################")
print("   # Initialize Azure Client                 ")
print("####################")

def init_clients():
    blob_service_client = BlobServiceClient.from_connection_string(AZURE_BLOB_CONNECTION_STRING)
    event_hub_client = EventHubProducerClient.from_connection_string(AZURE_EVENT_HUB_CONNECTION_STRING)
    openai_client=OpenAIClient(endpoint=AZURE_OPENAI_ENDPOINT, credential=AzureKeyCredential(AZURE_OPENAI_API_KEY))
    return blob_service_client, event_hub_client, openai_client

All the required setup is done. Now, we have to implement individual layers.
print("###############################")
print("   # Layer 1: Data Collection (IoT Sensor Simulation)       ")
print("###############################")



def send_sensor_data_to_iot_hub(sensor_data):
    device_client = IoTHubDeviceClient.create_from_connection_string(AZURE_IOT_HUB_CONNECTION_STRING)
    message = Message(sensor_data)
    device_client.send_message(message)
    print(f"Sent data to IoT Hub: {sensor_data}")

print("######################################")
print("   Layer 2: Data Ingestion Layer (Upload to Azure Blob Storage)          ")
print("######################################")


def store_data_in_blob(blob_service_client, file_name, content):
    blob_client = blob_service_client.get_blob_client(container=AZURE_STORAGE_CONTAINER, blob=file_name)
    blob_client.upload_blob(content, overwrite=True)
    print(f"Uploaded sensor data to Blob Storage: {file_name}")

print("#########################")
print("   Layer 3: Processing Layer (Text Analysis)    ")
print("#########################")


def send_data_to_event_hub(event_hub_client, data):
    event_data = EventData(data)
    with event_hub_client as client:
        client.send_batch([event_data])
    print(f"Sent data to Event Hub: {data}")

print("####################################")
print("   Layer 4: AI/ML Analysis (Predictive Insights with OpenAI)        ")
print("####################################")

def analyze_data_with_openai(openai_client, data):
    response = openai_client.completions.create(
        engine="text-davinci-003",
        prompt=f"Analyze the following IoT sensor data and predict possible equipment failures:\n\n{data}",
        max_tokens=150
    )
    prediction = response.choices[0].text.strip()
    print(f"Prediction: {prediction}")
    return prediction

print("####################")
print("   # Main Execution Workflow        ")
print("####################")

def main():
    # Initialize Azure Clients
    blob_service_client, event_hub_client, openai_client = init_clients()

    # Simulated Sensor Data for demonstrating the flow
    sensor_data = {
        "temperature": 85.5,
        "vibration": 0.03,
        "pressure": 101.2
    }
    sensor_data_str = str(sensor_data)

print("#############################")
print("   # Calling each layer to execute the work flow      ")
print("#############################")
    
    # Layer 1: Send Data to IoT Hub
    send_sensor_data_to_iot_hub(sensor_data_str)

    # Layer 2: Store Data in Blob Storage
    store_data_in_blob(blob_service_client, "sensor_data.json", sensor_data_str)

    # Layer 3: Stream Data to Event Hub
    send_data_to_event_hub(event_hub_client, sensor_data_str)

    # Layer 4: Analyze Data with OpenAI
    prediction = analyze_data_with_openai(openai_client, sensor_data_str)

    # Final Output
    print("### Final Prediction Output ###")
    print(f"Sensor Data: {sensor_data}")
    print(f"Prediction: {prediction}")

if __name__ == "__main__":
    main()


